In [1]:
from __future__ import print_function

# to be able to see plots
%matplotlib inline  
import matplotlib.pyplot as plt

import numpy as np

import sys
sys.path.append("../tools")

from tools import collage

# just to use a fraction of GPU memory 
# This is not needed on dedicated machines.
# Allows you to share the GPU.
# This is specific to tensorflow.
gpu_memory_usage=0.8 
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = gpu_memory_usage
set_session(tf.Session(config=config))



Using TensorFlow backend.


# Read dataset
Good corpus to start with is WIKI. A easy starting version can be downloaded from http://www.cs.upc.edu/~nlp/wikicorpus/.

In [2]:
with open('./en.wiki.txt', 'r') as f:
    data = f.read()
data = data[:50000000]
pos = np.random.randint(len(data)-500)
print(data[pos:pos+1000]) 

dataT = np.asarray(bytearray(data), dtype=np.uint8)    

 worn by the Javanese people of Indonesia.
Barong Tagalog, an embroidered formal garment of the Philippines;
Kimono, the traditional garments of Japan;
Ao dai, traditional garments of Vietnam;
Morning dress, a particular category of men's formal dress;
Topor, a type of conical headgear;
Seelai,Tamil Brides traditional formal wear;
traditionally worn by grooms as part of the Bengali Hindu wedding ceremony
Tuxedo;
Black tie, indicating dinner jacket in the UK;
White tie, indicating evening dress in the UK;
Sherwani, a long coat-like garment worn in South Asia;
Wedding veil;
Wedding dress;

Music.
Western weddings.
Music often played at western weddings includes a processional song for walking down the aisle (ex: Wedding March) and reception dance music. More at wedding music.

African weddings.
In traditional African weddings its a combining of two families.Traditional music throughout Africa is almost always functional; in other words, it is performed to mark a ritual such as a wedding.

# Create data generator

The generator creates mini-batches which of continuous sequences - that is: sequence 1 in batch t starts where sequence 1 ended in batch t-1.

reset() method restarts the sequences from new random positions.

The behavior is used in statefull behavior of the network during training.

In [6]:
class myGenerator(object):
    def __init__(self, data, length=48, batchSize=32):
        self.length = length
        self.batchSize = batchSize
        self.data = data
        self.positions = np.arange(0, data.shape[0], data.shape[0] / batchSize)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        return self.next()
    
    def next(self):
        d = []
        l = []
        for i in range(self.batchSize):
            p = self.positions[i]
            d.append(self.data[p:p+length])
            l.append(self.data[p+1:p+length+1])
            self.positions[i] = (self.positions[i] + length) % (self.data.shape[0] - 2*length)
            
        return np.stack(d), np.stack(l).reshape(self.batchSize, self.length, 1)
    
    def reset(self):
        self.positions = np.random.randint(self.data.shape[0]-10*self.length, size=self.positions.size)
    
batchSize = 32    
length = 48
generator = myGenerator(dataT, length, batchSize)

# Define net

In [7]:
from keras.layers import Embedding, CuDNNGRU, SimpleRNN, CuDNNLSTM
from keras.layers import Activation, BatchNormalization, Dense, Average, Maximum, Concatenate
from keras.models import Model
from keras import regularizers, initializers


def get_GRU_network(input_data, layer_cout, layer_dim, stateful=False):
    net = Embedding(256, output_dim=32)(input_data)
    net = BatchNormalization()(net)
    previous = []
    for i in range(layer_cout):
        net = CuDNNGRU(layer_dim, return_sequences=True, stateful=stateful, 
                       kernel_regularizer=regularizers.l2(0.000001),
                       recurrent_regularizer=regularizers.l2(0.000001))(net)
        previous.append(net)
        if len(previous) > 1:
            net1 = Average()(previous)
            net2 = Maximum()(previous)
            net = Concatenate()([net1, net2, net])
            
        net = Dense(layer_dim)(net)
        net = BatchNormalization()(net)
        net = Activation('relu')(net)
        
    net = Dense(256)(net)
    
    net = Activation('softmax')(net)

    return net


def get_GRU_simple_network(input_data, layer_cout, layer_dim, stateful=False):
    net = Embedding(256, output_dim=32)(input_data)
    net = BatchNormalization()(net)
    for i in range(layer_cout):
        net = CuDNNGRU(layer_dim, return_sequences=True, stateful=stateful)(net)
        net = BatchNormalization()(net)
       
    net = Dense(256)(net)
    net = Activation('softmax')(net)

    return net

In [8]:
from keras import optimizers
from keras.models import Model
from keras import losses
from keras import metrics
from keras.layers import Input

layerCount = 2
layerSize = 720

input_data = Input(batch_shape=(batchSize, length), name='data')
net = get_GRU_simple_network(input_data, layerCount, layerSize, stateful=True)
model = Model(inputs=[input_data], outputs=[net])

# model for text generation
input_data = Input(batch_shape=(1,1), name='data')
predictNet = get_GRU_simple_network(input_data, layerCount, layerSize, stateful=True)
predModel = Model(inputs=[input_data], outputs=[predictNet])

print('Model')
model.summary()


Model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
data (InputLayer)            (32, 48)                  0         
_________________________________________________________________
embedding_1 (Embedding)      (32, 48, 32)              8192      
_________________________________________________________________
batch_normalization_1 (Batch (32, 48, 32)              128       
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (32, 48, 720)             1628640   
_________________________________________________________________
batch_normalization_2 (Batch (32, 48, 720)             2880      
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (32, 48, 720)             3114720   
_________________________________________________________________
batch_normalization_3 (Batch (32, 48, 720)             2880      
____

In [10]:
model.compile(
    loss=losses.sparse_categorical_crossentropy, 
    optimizer=optimizers.Adam(lr=0.0002, clipnorm=5., clipvalue=1), # no good reason for clipnorm and clipvalue just experimenting
    metrics=[metrics.sparse_categorical_accuracy])

In [12]:
import keras
# This callback resets sequence with probability 10% after each batch
class My_Callback(keras.callbacks.Callback):
    def on_batch_end(self, batch, logs={}):
        if np.random.rand() > 0.9:
            self.model.reset_states()
            generator.reset()
        return
 
model.fit_generator(generator=generator, steps_per_epoch=1000, epochs=30, verbose=1, callbacks=[My_Callback()])
    


Epoch 1/30
1000/1000 [==============================] - 39s 39ms/step - loss: 2.3082 - sparse_categorical_accuracy: 0.4172
Epoch 2/30
1000/1000 [==============================] - 38s 38ms/step - loss: 1.8571 - sparse_categorical_accuracy: 0.4916
Epoch 3/30
1000/1000 [==============================] - 37s 37ms/step - loss: 1.7399 - sparse_categorical_accuracy: 0.5163
Epoch 4/30
1000/1000 [==============================] - 38s 38ms/step - loss: 1.6557 - sparse_categorical_accuracy: 0.5322
Epoch 5/30
1000/1000 [==============================] - 38s 38ms/step - loss: 1.6031 - sparse_categorical_accuracy: 0.5448
Epoch 6/30
1000/1000 [==============================] - 38s 38ms/step - loss: 1.5812 - sparse_categorical_accuracy: 0.5491
Epoch 7/30
1000/1000 [==============================] - 37s 37ms/step - loss: 1.5405 - sparse_categorical_accuracy: 0.5596
Epoch 8/30
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5332 - sparse_categorical_accuracy: 0.5607
Epoch 9/30
1000/

# Generate text

In [13]:
model.save_weights('model.mod')
predModel.load_weights('model.mod', by_name=False)
predModel.reset_states()

In [19]:
last = np.zeros(1, dtype=int).reshape(1,1)
last[0,0] = dataT[10]
predModel.reset_states()
startString = u'Start the string with this'
if startString:
    startString = [ord(x) for x in startString]
    last[0,0] = startString[0]
    print(chr(last[0,0]), end='')
    for i in startString[1:]:
        pred = predModel.predict(last)
        last[0,0] = i
        print(chr(last[0,0]), end='')

for i in range(10000):
    pred = predModel.predict(last)

    #print(pred)
    p = pred[0,0]**1.2
    p /= p.sum()    

    last[0,0] = np.random.choice(p.size, p=p)
    print(chr(last[0,0]), end='')
    
print()

Start the string with this new material objects or course critically attractive declarations and early powers.  The state of his army in Nova Scotia.

In 1942 the Language Hockey Days (420 km). Later, a clothing company based on the race of unebury. She has become or training pressure cartoons  up with Odin as a punk rocket.

On 1 Seoul Credits also studied lapse stations in the first time almost entire that where participated in the Bay and pretended the early 2001">
The also set up fuse of the passing rights and the working system entity. There were he published the wing in Port Star Wars who were ramined the string of the Katah (San Duble).







ENDOFARTICLE.
</doc>
<doc id="6606142" title="Nevakiah District" nonfiltered="923" processed="720" dbindex="1007056">

In October 1823 substrict has been streaking off.  Colonel and land visit the result for the two sites of Savoy broadcasts her to grow freedom of the painting and accts authorized quick product.

In 2003 the cartoon XI is 

Cannon came for the presence of its contract to the Longham River in 1823. Polish political particles when the new National Railways based on the Metropolitan Russian Francisco has strict purpose of reasons that I not the training is surrounded by range (or closed capitol).  This mapped players in death, all of which would have ever approved the series was such as "Bordean Bank". There are she noise the first character was in the time. The school drives and other four weapons, and slightly studyofs winter.  It is packed the playoffsis in service, Lichter and Mot�ch, Homereming and a status 2006 economy, in 1922, ahead 12.6 miles (25 km) centuries. Text, personal series were all closely theme in the strip was the can set up to 4 path (in which the Mexicano bookand). Small villages were born in two language and caring submarine war.

The company during his return to the piano series 308 papers fund not only eminent two men, asometimes may be a determined pain to what use.

Joshua Major M